# TP en Traitement Automatique du Langage Naturel: Classification de Sentiments sur des Critiques de Films

## Objectif
L'objectif de ce TP est de développer un système de classification de sentiments utilisant des critiques de films. Vous utiliserez un ensemble de données IMDb et appliqueront un modèle K-Nearest Neighbors (KNN) pour classer les critiques en catégories positives ou négatives.

## Partie 1: Traitement des Textes
1. **Preprocess**: Appliquer un preprocess si besoin
1. **Vectorisation**: Transformez les documents textuels en vecteurs numériques en utilisant `TfidfVectorizer`.

## Partie 3: Modélisation
1. **Construction du Modèle KNN**: Créez un modèle KNN
2. **Entraînement du Modèle**: Entraînez le modèle sur l'ensemble d'entraînement.

## Partie 4: Évaluation
1. **Prédiction et Classification**: Utilisez le modèle pour prédire les sentiments sur l'ensemble de test.
2. **Rapport de Classification**: Générez un rapport de classification pour évaluer la performance du modèle.

## Questions
1. Comment la réduction du nombre de caractéristiques (`max_features`) affecte-t-elle la performance du modèle ?
2. Quel impact a le choix du nombre de voisins dans KNN sur les résultats ?
3. Comparez les performances du modèle KNN avec un autre classificateur (par exemple, [Naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html) ou [SVM](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC). Lequel performe mieux et pourquoi ?
4. Le preprocessing améliore t-il la clasification ?

## Ressources
- [IMDb Dataset](https://huggingface.co/datasets/imdb)
- [Scikit-learn Documentation](https://scikit-learn.org/stable/)

In [ ]:
!pip install -q -U datasets scikit-learn

In [1]:
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

# On split plusieur fois le dataset afin de réduire le temps de calcule
dataset = load_dataset("imdb", split="train")
dataset = dataset.train_test_split(stratify_by_column="label", test_size=0.3, seed=42)
dataset = dataset['train'].train_test_split(stratify_by_column="label", test_size=0.3, seed=42)

Using the latest cached version of the module from /home/blancsw/.cache/huggingface/modules/datasets_modules/datasets/imdb/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0 (last modified on Wed Dec 20 10:48:44 2023) since it couldn't be found locally at imdb, or remotely on the Hugging Face Hub.


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
set(dataset['train']['label'])

{0, 1}

In [ ]:
# Vectorization of text documents
vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(dataset['train']['text'])
X_test = vectorizer.transform(dataset['test']['text'])

# KNN Classifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, dataset['train']['text'])

# Test the classifier
y_pred = knn.predict(X_test)

print(classification_report(y_test, y_pred))